In [1]:
%pip install -U sentence-transformers

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB 682.7 kB/s eta 0:00:24
   ---------------------------------------- 0.2/15.8 MB 2.2 MB/s eta 0:00:08
   - -------------------------------------- 0.5/15.8 MB 4.6 MB/s eta 0:00:04
   -- ------------------------------------- 0.9/15.8 MB 5.4 MB/s eta 0:00:03
   --- ------------------------------------ 1.4/15.8 MB 6.4 MB/s eta 0:00:03
   --- ------------------------------------ 1.5/15.8 MB 6.5 MB/s eta 0:00:03
   --- ------------------------------------ 1.5/15.8 MB 6.5 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/15.8 MB 5.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.9/15.8 MB 4.6 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/15.8 MB 4.7 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/15.8 MB 4.7 MB/s eta 0:00:03
   ----- -

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

In [ ]:
hs_tree = pd.read_csv("../data/hs_tree_cleaned.csv")
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = np.load("../data/processed/hs_tree_embeddings.npy")

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/hs_tree_embeddings.npy'

In [9]:
index = faiss.read_index("../models/faiss_index.index")

In [14]:
query = "LED TV with 4K display"
query_embedding = model.encode([query])

NameError: name 'model' is not defined

In [6]:
D, I = index.search(np.array(query_embedding).astype("float32"), k=3)

In [8]:
print(len(embeddings), len(hs_tree))


6939 6939


In [10]:
top_3_results = [
    (hs_tree.iloc[int(idx)]['HS6'], 1 - D[0][j]) 
    for j, idx in enumerate(I[0])
]

In [15]:
def log_search_results(query, results):
    print(f"\nQuery: {query}")
    for hs6_code, score in results:
        print(f"HS6: {hs6_code} | Confidence: {score:.4f}")


In [16]:
log_search_results(query, top_3_results)


Query: LED TV with 4K display
HS6: nan | Confidence: -0.2463
HS6: 854011.0 | Confidence: -0.2463
HS6: 854012.0 | Confidence: -0.3249


In [17]:
search_log = []

def log_search_results_to_csv(query, results):
    for hs6_code, confidence in results:
        search_log.append({
            "query": query,
            "hs6_result": hs6_code,
            "confidence_score": round(confidence, 4)
        })

In [30]:
query = "Fresh apples"
query_embedding = model.encode([query])[0]

D, I = index.search(np.array([query_embedding]), k=3)

top_3_results = [
    (hs_tree.iloc[int(idx)]['HS6'], 1 - D[0][j]) 
    for j, idx in enumerate(I[0])
]

# Log results
log_search_results_to_csv(query, top_3_results)


In [31]:
df_log = pd.DataFrame(search_log)
df_log.to_csv("../outputs/search_results_log.csv", index=False)
